In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("WSDM User Logs Processing").getOrCreate() 
df_validation = spark.read.csv('./data/sample_submission_v2.csv', inferSchema=True, header=True)
df_userlogs = spark.read.csv('./data/user_logs_v2.csv', inferSchema=True, header=True)
# Limit userlog 'msno' column to only include those 'msno' values present in 'df_validation', so we only have the logs 
# of relevant users (in this case, the users in the validation set). This will take a while, as will all Spark SQL calls.
df_validation_userlogs = df_userlogs.join(df_validation, 'msno', how='inner')
# Clear up some memory
del df_userlogs

In [2]:
# Now, to aggregate over the relevant user log data
AGG_COLS = ['num_100', 'num_25', 'num_50', 'num_75', 'num_985', 'num_unq', 'total_secs']
AGG_FNS = ['avg', 'max', 'min', 'sum', 'stddev']
build_agg_dict = lambda agg_name: dict((cname, agg_name) for cname in AGG_COLS)
dfs_agg = []

for agg_name in AGG_FNS:
    agg_dict = build_agg_dict(agg_name)
    df_agg = df_validation_userlogs.groupby('msno').agg(agg_dict)
    dfs_agg.append(df_agg)
    
# Clear up some memory
del df_validation_userlogs

for df_agg in dfs_agg:
    df_validation = df_validation.join(df_agg, 'msno', how='inner')
    del df_agg

assert len(df_validation.columns) >= 30

In [4]:
# Now we write the validation dataframe to a directory called 'validation_data'. Since the Spark execution
# model performs everything in parts, I've written a method in utils.py to compile all the parts' individual
# csv files into one larger csv file. This file will only be ~300MB and will fit easily into memory so we
# can do all our regular tricks with pandas.
import os
import pandas as pd

from utils import compile_csv_parts_to_larger_csv

VALIDATION_DATA_PATH = './data/validation_ulog_agg'
VALIDATION_CSV_PATH = './data/validation_ulog_agg_v2.csv'

df_validation.write.csv(VALIDATION_DATA_PATH, header=True)
assert os.path.isdir(VALIDATION_DATA_PATH)
del df_validation
compile_csv_parts_to_larger_csv(csv_parts_path=VALIDATION_DATA_PATH, to_write_path=VALIDATION_CSV_PATH)
assert os.path.isfile(VALIDATION_CSV_PATH)

# Sanity check
df_validation = pd.read_csv(VALIDATION_CSV_PATH)
df_validation.head()

Now processing file name ./data/validation_ulog_agg/part-00000-d90d4868-b198-41e0-aabf-5267b96e3627-c000.csv...
Now processing file name ./data/validation_ulog_agg/part-00001-d90d4868-b198-41e0-aabf-5267b96e3627-c000.csv...
Now processing file name ./data/validation_ulog_agg/part-00002-d90d4868-b198-41e0-aabf-5267b96e3627-c000.csv...
Now processing file name ./data/validation_ulog_agg/part-00003-d90d4868-b198-41e0-aabf-5267b96e3627-c000.csv...
Now processing file name ./data/validation_ulog_agg/part-00004-d90d4868-b198-41e0-aabf-5267b96e3627-c000.csv...
Now processing file name ./data/validation_ulog_agg/part-00005-d90d4868-b198-41e0-aabf-5267b96e3627-c000.csv...
Now processing file name ./data/validation_ulog_agg/part-00006-d90d4868-b198-41e0-aabf-5267b96e3627-c000.csv...
Now processing file name ./data/validation_ulog_agg/part-00007-d90d4868-b198-41e0-aabf-5267b96e3627-c000.csv...
Now processing file name ./data/validation_ulog_agg/part-00008-d90d4868-b198-41e0-aabf-5267b96e3627-c000

Now processing file name ./data/validation_ulog_agg/part-00074-d90d4868-b198-41e0-aabf-5267b96e3627-c000.csv...
Now processing file name ./data/validation_ulog_agg/part-00075-d90d4868-b198-41e0-aabf-5267b96e3627-c000.csv...
Now processing file name ./data/validation_ulog_agg/part-00076-d90d4868-b198-41e0-aabf-5267b96e3627-c000.csv...
Now processing file name ./data/validation_ulog_agg/part-00077-d90d4868-b198-41e0-aabf-5267b96e3627-c000.csv...
Now processing file name ./data/validation_ulog_agg/part-00078-d90d4868-b198-41e0-aabf-5267b96e3627-c000.csv...
Now processing file name ./data/validation_ulog_agg/part-00079-d90d4868-b198-41e0-aabf-5267b96e3627-c000.csv...
Now processing file name ./data/validation_ulog_agg/part-00080-d90d4868-b198-41e0-aabf-5267b96e3627-c000.csv...
Now processing file name ./data/validation_ulog_agg/part-00081-d90d4868-b198-41e0-aabf-5267b96e3627-c000.csv...
Now processing file name ./data/validation_ulog_agg/part-00082-d90d4868-b198-41e0-aabf-5267b96e3627-c000

Now processing file name ./data/validation_ulog_agg/part-00149-d90d4868-b198-41e0-aabf-5267b96e3627-c000.csv...
Now processing file name ./data/validation_ulog_agg/part-00150-d90d4868-b198-41e0-aabf-5267b96e3627-c000.csv...
Now processing file name ./data/validation_ulog_agg/part-00151-d90d4868-b198-41e0-aabf-5267b96e3627-c000.csv...
Now processing file name ./data/validation_ulog_agg/part-00152-d90d4868-b198-41e0-aabf-5267b96e3627-c000.csv...
Now processing file name ./data/validation_ulog_agg/part-00153-d90d4868-b198-41e0-aabf-5267b96e3627-c000.csv...
Now processing file name ./data/validation_ulog_agg/part-00154-d90d4868-b198-41e0-aabf-5267b96e3627-c000.csv...
Now processing file name ./data/validation_ulog_agg/part-00155-d90d4868-b198-41e0-aabf-5267b96e3627-c000.csv...
Now processing file name ./data/validation_ulog_agg/part-00156-d90d4868-b198-41e0-aabf-5267b96e3627-c000.csv...
Now processing file name ./data/validation_ulog_agg/part-00157-d90d4868-b198-41e0-aabf-5267b96e3627-c000

,avg(num_100),sum(total_secs),sum(num_50),min(num_75),avg(num_50),avg(num_985),min(total_secs),min(num_985),sum(num_25),min(num_unq),...,sum(num_unq),max(num_unq),max(num_50),stddev(num_25),stddev(num_unq),stddev(num_100),stddev(num_50),stddev(total_secs),stddev(num_75),stddev(num_985)
0,9.000000,2362.332,1,0,1.000000,0.000000,2362.332,0,14,24,...,24,24,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,12.000000,59013.159,18,0,1.058824,0.470588,119.440,0,8,1,...,149,28,12,1.007326,7.964997,13.057565,2.882503,3924.399063,0.528594,1.007326
2,30.217391,196442.810,58,0,2.521739,0.869565,864.783,0,970,4,...,1663,161,9,36.833350,41.307533,14.478688,2.212969,4108.320638,1.202764,0.868873
3,34.966667,264692.811,25,0,0.833333,1.033333,1111.770,0,345,1,...,737,63,8,12.808268,16.564781,21.350092,1.683251,5464.339845,0.808717,1.299425
4,48.458333,296370.987,27,0,1.125000,0.583333,646.093,0,100,3,...,1021,168,14,14.330468,45.053571,51.425741,2.908944,13264.404016,0.884652,0.974308
